# Tensors

In [1]:
import torch

In [2]:
a = torch.ones(3)
print(a[0]," ",float(a[0]))

tensor(1.)   1.0


In [3]:
points = torch.tensor([[1.,1.],[2.,2.],[3.,3.]])
points

tensor([[1., 1.],
        [2., 2.],
        [3., 3.]])

In [4]:
points.shape

torch.Size([3, 2])

## Named tensors

In [5]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [6]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [7]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [8]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
unsqueezed_weights
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [9]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [10]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [11]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [12]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

## Tensor element types

In [13]:
# float32 and int64 are the most used
x_t = torch.tensor([1,2,3],dtype= torch.uint8)
x_t

tensor([1, 2, 3], dtype=torch.uint8)

In [14]:
double_points = x_t.double()
# or double_points = x_t.to(torch.double)
double_points

tensor([1., 2., 3.], dtype=torch.float64)

## The tensor API

In [15]:
y_t = torch.tensor([2.2,-2.,-4.])
y_t, abs(y_t), y_t.acos(), y_t.log(), y_t.log2(), y_t.neg(), y_t.pow(2), y_t.round(), y_t.sigmoid(), y_t.tanh()

(tensor([ 2.2000, -2.0000, -4.0000]),
 tensor([2.2000, 2.0000, 4.0000]),
 tensor([nan, nan, nan]),
 tensor([0.7885,    nan,    nan]),
 tensor([1.1375,    nan,    nan]),
 tensor([-2.2000,  2.0000,  4.0000]),
 tensor([ 4.8400,  4.0000, 16.0000]),
 tensor([ 2., -2., -4.]),
 tensor([0.9002, 0.1192, 0.0180]),
 tensor([ 0.9757, -0.9640, -0.9993]))

In [16]:
z_t = torch.tensor([2.,-2.,-4.])
z_t * y_t, z_t.mul(y_t), 

(tensor([ 4.4000,  4.0000, 16.0000]), tensor([ 4.4000,  4.0000, 16.0000]))

In [17]:
z_t.max(), z_t.mean(), z_t.std(), z_t.norm()

(tensor(2.), tensor(-1.3333), tensor(3.0551), tensor(4.8990))

In [18]:
mat_t = torch.tensor([
    [0,3,4],
    [4,5,6],
    [6,2,4]
],dtype = torch.float32)

mat2_t = torch.tensor([
    [1,0,0],
    [1,1,0],
    [0,0,1]
],dtype = torch.float32)
mat_t.trace(), mat_t.det(), mat_t.lu(), mat_t.matmul(mat2_t)

(tensor(9.),
 tensor(-28.0000),
 (tensor([[6.0000, 2.0000, 4.0000],
          [0.6667, 3.6667, 3.3333],
          [0.0000, 0.8182, 1.2727]]),
  tensor([3, 2, 3], dtype=torch.int32)),
 tensor([[3., 3., 4.],
         [9., 5., 6.],
         [8., 2., 4.]]))

In [19]:
mat_t.storage()

 0.0
 3.0
 4.0
 4.0
 5.0
 6.0
 6.0
 2.0
 4.0
[torch.FloatStorage of size 9]

### in place _

In [20]:
r = torch.ones(3, 2)
r.zero_()
r

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor metadata: Size, offset, and stride

In [21]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]

second_point.storage_offset(), second_point.size(),second_point.stride(), points.stride()

(2, torch.Size([2]), (1,), (2, 1))

In [22]:
# changing the subtensor will have a side effect on the original tensor:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points, id(points.storage()) == id(second_point.storage())

(tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]),
 True)

In [23]:
# clone 
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
third_point = points[1].clone()
third_point[0] = 10.0
points, id(points.storage()) == id(third_point.storage())

(tensor([[4., 1.],
         [5., 3.],
         [2., 1.]]),
 True)

### Contiguous tensors

In [24]:
points.is_contiguous(), points[:,0].is_contiguous()

(True, False)

In [25]:
pointss = torch.tensor([[4, 1], [5, 3], [2, 1]])
points_ts = points.t()
points_ts
points_t_cont = points_ts.contiguous()
pointss.storage()[0]=8
pointss,points_t_cont,points_t_cont.storage(),pointss.storage(),id(points_t_cont.storage()),id(pointss.storage()) ,id(points_t_cont.storage())== id(pointss.storage()) #wtffffff!!!!!!!!!! because it gives the address of the fucntion if we store storage in  variable it will give us the correct result

(tensor([[8, 1],
         [5, 3],
         [2, 1]]),
 tensor([[4., 5., 2.],
         [1., 3., 1.]]),
  4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.FloatStorage of size 6],
  8
  1
  5
  3
  2
  1
 [torch.LongStorage of size 6],
 140460506821568,
 140460506821568,
 True)

In [27]:
points_t_cont_storage =points_t_cont.storage()
pointss_storage = pointss.storage()
pointss,points_t_cont,points_t_cont.storage(),pointss.storage(),id(points_t_cont_stotage),id(pointss_storage) ,id(points_t_cont_storage)== id(pointss_storage) #wtffffff!!!!!!!!!! because it gives the address of the fucntion if we store storage in  variable it will give us the correct result

(tensor([[8, 1],
         [5, 3],
         [2, 1]]),
 tensor([[4., 5., 2.],
         [1., 3., 1.]]),
  4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.FloatStorage of size 6],
  8
  1
  5
  3
  2
  1
 [torch.LongStorage of size 6],
 140460506822656,
 140460506218624,
 False)

## 3.10 NumPy interoperability

In [30]:
pb = torch.ones(3, 4)
points_np = pb.numpy()
points_np, torch.from_numpy(points_np)

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]))

## Serializing tensors

In [31]:
torch.save(points, './ourpoints.t')

In [32]:
with open('./ourpoints.t','wb') as f:
    torch.save(points, f)

In [33]:
with open('./ourpoints.t','rb') as f: 
    points = torch.load(f)

##### Serializing to HDF5 with h5py

In [35]:
import h5py

In [36]:
f = h5py.File('./ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [38]:
f = h5py.File('./ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]# only these rows are loded
last_points

array([[5., 3.],
       [2., 1.]], dtype=float32)